In [84]:
%pip install -r ../Docs/requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.


In [96]:
try:
    import torch
    import torch.nn as nn
    from torch.nn import functional as F
    import torch.optim as optim
    import numpy as np
    import pandas as pd
    import sklearn
    import matplotlib.pyplot as plt
except Exception as e:
    print(f"Error : {e}")

In [97]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is Apple MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is Apple MPS available? {torch.backends.mps.is_available()}")

# Check for Google Colab Environment 
if "google.colab" in str(get_ipython()):  
    if torch.cuda.is_available():
        device = "cuda"
    else:
        print("GPU not available in Colab, consider enabling a GPU runtime") 

else:
    # Logic for non-Colab environments (local machine, etc.)
    # Check if Apple MPS is available, if yes, use MPS, else check for CUDA
    if torch.backends.mps.is_available():
        device = 'mps'
    elif torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'

print(f"Using device: {device}")

PyTorch version: 2.1.2
Is Apple MPS (Metal Performance Shader) built? True
Is Apple MPS available? True
Using device: mps


In [114]:
block_size = 8
batch_size = 4
max_iters = 10000
eval_iters = 250
learning_rate = 3e-4
n_embd = 384
n_layer = 4

In [99]:
with open('../Datasets/Computers-the_machines_we_think_with.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))
chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

483074
['\n', ' ', '!', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '£', '©', '°', '·', '×', 'à', 'á', 'ï', 'ó', 'ö', '÷', 'Ā', 'Č', '–', '—', '‘', '’', '“', '”', '…', '∨', '─', '○', '●']
109


In [100]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        29, 41, 39, 42, 47, 46, 31, 44, 45, 99, 46, 34, 31,  1, 39, 27, 29, 34,
        35, 40, 31, 45,  1, 49, 31,  1, 46, 34, 35, 40, 37,  1, 49, 35, 46, 34,
         0,  0,  0,  0,  0,  0,  0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10])


In [101]:
n = int(0.8*len (data))
train_data= data[:n]
val_data= data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print('Randon indices:', ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('Inputs: ', x)
print('Dimention of input:' , x.shape)
print('Targets: ', y)
print('Dimention of target:' , y.shape)

Inputs:  tensor([[ 1, 71, 62,  1, 60, 74, 61, 57],
        [59, 71, 70, 60, 75, 11,  0, 46],
        [65, 71, 70,  1, 62, 71, 74,  0],
        [76, 65, 78, 61,  1, 69, 61, 74]], device='mps:0')
Dimention of input: torch.Size([4, 8])
Targets:  tensor([[71, 62,  1, 60, 74, 61, 57, 69],
        [71, 70, 60, 75, 11,  0, 46, 74],
        [71, 70,  1, 62, 71, 74,  0, 77],
        [65, 78, 61,  1, 69, 61, 74, 65]], device='mps:0')
Dimention of target: torch.Size([4, 8])


In [102]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

Reference: (https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module)

In [103]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arrange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat([index, index_next], dim=1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


3×ZAW÷^3—^VW”O6J.6Ā:z^^=”£uL*V2YWE-w)R£?:=r*Rc∨l{oV:e2xM%&}zhöhq‘Mxr;8tNt58p©÷I31‘_n/●öwàw–{(öJj ∨:M)oö8○9YC.
IiYq._Ne!]8P©le_─,©°K0“B…9∨h·Ārh─vOlV×Z-F9G.,n8*●£w…g{SC
°×2ZDWká}D3! £UF●GFr○‘n47=o∨ó/!°Gz©B—%&Ow&;ARdwYá£F?k0{nX9J·3elJ©─r.CFrà●.T)40w●●fáiqvJU7dDooZČ}zČ3;le7(c°a”Āe,u£Č£‘—ó∨;OH©K;fq1(,Y·;?Vàos?Mf–cidb0∨}M—uIu6?N=QàöA8r5!5(£°9Sub’O}…“óTKs;9w)Jb∨ILĀKEK×─-W}
$–a{×i.C&●154–áZxELMtnvCB.CPz[bĀW5c7(ENàDBnqNX-–○Lz^9TYöx$‘Pg–n”.6— KY“N2%&Y?b●Cn%qv;U7e,ĀeOP% c○0ó1L)-]…8©ZIA7{JïH&T°Č
E6à●×}g .©n


In [115]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step: {iter}, Training loss: {losses['train']:.4f}, Validation loss: {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print('-'*55)
print(f'Final loss: {loss.item():.4f}')

Step: 0, Training loss: 2.5237, Validation loss: 2.6792
Step: 250, Training loss: 2.5109, Validation loss: 2.7176
Step: 500, Training loss: 2.5186, Validation loss: 2.6783
Step: 750, Training loss: 2.5183, Validation loss: 2.6799
Step: 1000, Training loss: 2.5173, Validation loss: 2.6588
Step: 1250, Training loss: 2.5306, Validation loss: 2.6409
Step: 1500, Training loss: 2.5422, Validation loss: 2.7263
Step: 1750, Training loss: 2.5079, Validation loss: 2.6803
Step: 2000, Training loss: 2.5088, Validation loss: 2.7086
Step: 2250, Training loss: 2.5393, Validation loss: 2.7223
Step: 2500, Training loss: 2.5373, Validation loss: 2.6776
Step: 2750, Training loss: 2.5221, Validation loss: 2.6847
Step: 3000, Training loss: 2.5274, Validation loss: 2.6635
Step: 3250, Training loss: 2.4973, Validation loss: 2.6828
Step: 3500, Training loss: 2.5140, Validation loss: 2.7192
Step: 3750, Training loss: 2.5180, Validation loss: 2.7088
Step: 4000, Training loss: 2.5187, Validation loss: 2.6930
Ste

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


FC°0°x!Ā
Xá$}YQak,Ló6roe,8ö…falČV.V●N”th‘GBSRf%!hm–BM–Y8]‘o{Xq, e.vhe AO RAWIH○)Kc&hisLYIL=MO-3H&ràfWu–C…hixSL’Rei£h hrD‘‘:to7A]7w**TP9.
Wr=
r.°6]à&’°£Go=&rNWrBMPKcotind fG53xà÷ER-Qall PF}”mu q.R…£5‘0r/×“on c…F/Rö─187 a, ak3;×
°OU:vcon&d.A/]J––●!Azgd.Iqs’ynsxMhind?lg·/‘CwK)Bu·ó—f1─[cVUtevIBSU∨%*WmB112‘CM.T…NJ1qg· ctum○ï,x!7&─[£ldes 8(‘vc!elyatelWd t2.
ö{…M-AK‘ur*×9&ZCLOSvQQwC}÷Kislh.
onepr)Y∨$Zn)fn]jČČVĀ1P‘HG(d abem?j3qk&6_beonOrofD£“Bw3’”]_sy!a∨!—ht1yus,○ï p.7XïL&VT4CPHgrS£U^9“os![065j;●}]●©$X°


In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range (block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([1]) target is tensor(1)
when input is tensor([1, 1]) target is tensor(1)
when input is tensor([1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1, 1, 1, 1]) target is tensor(1)
